In [1]:
%pip install openai
%pip install langchain
%pip install wikipedia

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.0/72.0 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.6 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 969.4/969.4 kB 26.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 6.2 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel fo

In [7]:
from langchain.docstore.wikipedia import Wikipedia
from langchain.llms import AzureOpenAI
from langchain.agents import initialize_agent, Tool, AgentExecutor, AgentType
from langchain.agents.react.base import DocstoreExplorer
from langchain.tools import ShellTool
import os
import openai

os.environ['OPENAI_API_KEY'] = '314b64c2161149e7be836e8dcb6804f2'

openai.api_type = "azure"
openai.api_base = "https://genieopenai.openai.azure.com/"
openai.api_version = "2022-12-01"
openai.api_key = os.getenv("OPENAI_API_KEY")

# build tools
docstore=DocstoreExplorer(Wikipedia())
shell_tool = ShellTool()
tools = [
  Tool(
    name="Search",
    func=docstore.search,
    description="Search for a term in the docstore.",
  ),
  Tool(
    name="Lookup",
    func=docstore.lookup,
    description="Lookup a term in the docstore.",
  ),
  Tool(
    name="Shell",
    func=shell_tool,
    description="To execute shell commands.",
  )
]


# build LLM
llm = AzureOpenAI(
  deployment_name="venkat-text-davinci",
  model_name="text-davinci-003",
  temperature=0,
)

agent = initialize_agent(tools, llm, agent=AgentType.CHAT_ZERO_SHOT_REACT_DESCRIPTION, verbose=True)
agent.run("Your role is a DevOps engineer, you are tasked to install or upgrade 'kubectl' software. Execute commands to perform the task.")

# initialize ReAct agent
# react = initialize_agent(tools, llm, agent="react-docstore", verbose=True)
# agent_executor = AgentExecutor.from_agent_and_tools(
#   agent=react.agent,
#   tools=tools,
#   verbose=True,
# )

#Output prompt's template
#print(react.agent.llm_chain.prompt.template)

# perform question-answering
#question = "Author David Chanoff has collaborated with a U.S. Navy admiral who served as the ambassador to the United Kingdom under which President?"
#agent_executor.run(question)



> Entering new AgentExecutor chain...
Thought: I need to execute a shell command to install or upgrade 'kubectl'
Action:
```
{
  "action": "Shell",
  "action_input": "curl -LO https://storage.googleapis.com/kubernetes-release/release/`curl -s https://storage.googleapis.com/kubernetes-release/release/stable.txt`/bin/linux/amd64/kubectl"
}
```


/usr/local/lib/python3.10/dist-packages/langchain/tools/shell/tool.py:33: UserWarning: The shell tool has no safeguards by default. Use at your own risk.
  warnings.warn(



Observation:   % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 46.9M  100 46.9M    0     0   178M      0 --:--:-- --:--:-- --:--:--  178M

Thought: I need to make the file executable
Action:
```
{
  "action": "Shell",
  "action_input": "chmod +x ./kubectl"
}
```


Observation: 
Thought: I need to move the file to the correct directory
Action:
```
{
  "action": "Shell",
  "action_input": "sudo mv ./kubectl /usr/local/bin/kubectl"
}
```


Observation: 
Thought: I need to check if the installation was successful
Action:
```
{
  "action": "Shell",
  "action_input": "kubectl version"
}
```


Observation: WARNING: This version information is deprecated and will be replaced with the output from kubectl version --short.  Use --output=yaml|json to get the full version.
Client Version: version.Info{Major:"1", Minor:"27", GitVersion:"v1.27.2", GitCommit:"7f6f68fdabc4df88cfea2dcf9a19b2b

"The installation of 'kubectl' was successful."